# Vector fields on the 3-sphere

This worksheet demonstrates a few capabilities of
[SageManifolds](http://sagemanifolds.obspm.fr) (version 1.0, as included in SageMath 7.5)
on the example of the 3-dimensional sphere, $\mathbb{S}^3$.

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v1.0/SM_sphere_S3_vectors.ipynb) to download the worksheet file (ipynb format). To run it, you must start SageMath with the Jupyter notebook, via the command `sage -n jupyter`

*NB:* a version of SageMath at least equal to 7.5 is required to run this worksheet:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [ ]:
%display latex

We also define a viewer for 3D plots (use `'threejs'` or `'jmol'` for interactive 3D graphics):

In [ ]:
viewer3D = 'threejs' # must be 'threejs', jmol', 'tachyon' or None (default)

## $\mathbb{S}^3$ as a 3-dimensional differentiable manifold

We start by declaring $\mathbb{S}^3$ as a differentiable manifold of dimension 3 over $\mathbb{R}$:

In [ ]:
S3 = Manifold(3, 'S^3', latex_name=r'\mathbb{S}^3', start_index=1)

The first argument, `3`, is the dimension of the manifold, while the second argument is the symbol used to label the manifold, with the LaTeX output specified by the argument `latex_name`. The argument `start_index` sets the index range to be used on the manifold for labelling components w.r.t. a basis or a frame: `start_index=1` corresponds to $\{1,2,3\}$; the default value is `start_index=0`, yielding to $\{0,1,2\}$.

In [ ]:
print(S3)

In [ ]:
S3

### Coordinate charts on $\mathbb{S}^3$

The 3-sphere cannot be covered by a single chart. At least two charts are necessary, for instance the charts associated with the stereographic projections from two distinct points, $N$ and $S$ say,
which we may call the *North pole* and the *South pole* respectively. Let us introduce the open subsets covered by these two charts: 
$$ U := \mathbb{S}^3\setminus\{N\} $$  
$$ V := \mathbb{S}^3\setminus\{S\} $$

In [ ]:
U = S3.open_subset('U') ; print(U)

In [ ]:
V = S3.open_subset('V') ; print(V)

We declare that $\mathbb{S}^3 = U \cup V$:

In [ ]:
S3.declare_union(U, V)

Then we introduce the stereographic chart on $U$, denoting by $(x,y,z)$ the coordinates resulting from the stereographic projection from the North pole onto the equatorial plane:

In [ ]:
stereoN.<x,y,z> = U.chart()
stereoN

In [ ]:
stereoN.coord_range()

Similarly, we introduce on $V$ the coordinates $(x',y',z')$ corresponding to the stereographic projection from the South pole onto the equatorial plane:

In [ ]:
stereoS.<xp,yp,zp> = V.chart("xp:x' yp:y' zp:z'")
stereoS

In [ ]:
stereoS.coord_range()

We have to specify the **transition map** between the charts `stereoN` = $(U,(x,y,z))$ and `stereoS` = $(V,(x',y',z'))$; it is given by the standard inversion formulas:

In [ ]:
r2 = x^2+y^2+z^2
stereoN_to_S = stereoN.transition_map(stereoS, 
                                      (x/r2, y/r2, z/r2), 
                                      intersection_name='W',
                                      restrictions1= x^2+y^2+z^2!=0, 
                                      restrictions2= xp^2+yp^2+zp^2!=0)
stereoN_to_S.display()

In the above declaration, `'W'` is the name given to the open subset where the two charts overlap: $W := U\cap V$, the condition $x^2+y^2+z^2\not=0$  defines $W$ as a subset of $U$, and the condition $x'^2+y'^2+z'^2\not=0$ defines $W$ as a subset of $V$.

The inverse coordinate transformation is computed by means of the method `inverse()`:

In [ ]:
stereoS_to_N = stereoN_to_S.inverse()
stereoS_to_N.display()

Note that the situation is of course perfectly symmetric regarding the coordinates $(x,y,z)$ and $(x',y',z')$.

At this stage, the user's atlas has four charts:

In [ ]:
S3.atlas()

For future reference, we store $W=U\cap V$ into a Python variable:

In [ ]:
W = U.intersection(V)
print(W)

### The North and South poles

$N$ is the point of $V$ of stereographic coordinates $(x',y',z')=(0,0,0)$:

In [ ]:
N = V((0,0,0), chart=stereoS, name='N')
print(N)

while $S$ is the point of U of stereographic coordinates $(x,y,z)=(0,0,0)$:

In [ ]:
S = U((0,0,0), chart=stereoN, name='S')
print(S)

We have of course

In [ ]:
all([N not in U, N in V, S in U, S not in V])

## Embedding of $\mathbb{S}^3$ into $\mathbb{R}^4$

Let us first declare $\mathbb{R}^4$ as a 4-dimensional manifold covered by a single chart (the so-called **Cartesian coordinates**):

In [ ]:
R4 = Manifold(4, 'R^4', r'\mathbb{R}^4')
X4.<T,X,Y,Z> = R4.chart()
X4

The embedding of $\mathbb{S}^3$ into $\mathbb{R}^4$ is then defined by the standard formulas relating the stereographic coordinates to the ambient Cartesian ones when considering a **stereographic projection** from the point $(-1,0,0,0)$ to the equatorial plane $T=0$:

In [ ]:
rp2 = xp^2 + yp^2 + zp^2
Phi = S3.diff_map(R4, {(stereoN, X4): 
                       [(1-r2)/(r2+1), 2*x/(r2+1), 
                        2*y/(r2+1), 2*z/(r2+1)],
                       (stereoS, X4):
                       [(rp2-1)/(rp2+1), 2*xp/(rp2+1), 
                        2*yp/(rp2+1), 2*zp/(rp2+1)]},
                  name='Phi', latex_name=r'\Phi')
Phi.display()

### Projections of $\mathbb{R}^4$ to $\mathbb{S}^3$

In [ ]:
R4N = R4.open_subset('R4N', latex_name=r'\mathbb{R}^4_N', 
                     coord_def={X4: T!=-1})
X4N = X4.restrict(R4N)

In [ ]:
ProjN = R4N.diff_map(U, {(X4N, stereoN): 
                         [X/(1+T), Y/(1+T), Z/(1+T)]},
                     name='P_N', latex_name=r'\Pi_N')
ProjN.display()

In [ ]:
R4S = R4.open_subset('R4S', latex_name=r'\mathbb{R}^4_S', 
                     coord_def={X4: T!=1})
X4S = X4.restrict(R4S)

In [ ]:
ProjS = R4S.diff_map(V, {(X4S, stereoS): 
                         [X/(1-T), Y/(1-T), Z/(1-T)]},
                     name='P_S', latex_name=r'\Pi_S')
ProjS.display()

In [ ]:
var('a b c', domain='real')
p = S3((1+a^2,b,c), chart=stereoN)
stereoN(p)

In [ ]:
all([p in U, p in V])

In [ ]:
all([ProjN(Phi(p)) == p, ProjS(Phi(p)) == p])

In [ ]:
p = S3((1+a^2,b,c), chart=stereoS)
all([ProjN(Phi(p)) == p, ProjS(Phi(p)) == p])

In [ ]:
q = R4((sqrt(3)/2, sin(a)*cos(b)/2, sin(a)*sin(b)/2, cos(a)/2))
X4(q)

In [ ]:
all([q in R4N, q in R4S])

In [ ]:
all([Phi(ProjN(q)) == q, Phi(ProjS(q)) == q])

## Hyperspherical coordinates

In [ ]:
A = W.open_subset('A', coord_def={stereoN.restrict(W): (y!=0, x<0), 
                                  stereoS.restrict(W): (yp!=0, xp<0)})
print(A)

In [ ]:
spher.<ch,th,ph> = A.chart(r'ch:(0,pi):\chi th:(0,pi):\theta ph:(0,2*pi):\phi')
spher

In [ ]:
den = 1 + cos(ch)
spher_to_stereoN = spher.transition_map(stereoN.restrict(A), 
                                        (sin(ch)*sin(th)*cos(ph)/den,
                                         sin(ch)*sin(th)*sin(ph)/den,
                                         sin(ch)*cos(th)/den))
spher_to_stereoN.display()

In [ ]:
spher_to_stereoN.set_inverse(2*atan(sqrt(x^2+y^2+z^2)),
                             atan2(sqrt(x^2+y^2), z),
                             atan2(-y, -x)+pi,
                             verbose=True)

In [ ]:
spher_to_stereoN.inverse().display()

In [ ]:
spher_to_stereoS = stereoN_to_S.restrict(A) * spher_to_stereoN
spher_to_stereoS.display()

In [ ]:
stereoS_to_spher = spher_to_stereoN.inverse() * stereoS_to_N.restrict(A)
stereoS_to_spher.display()

In [ ]:
Phi.display(stereoN.restrict(A), X4)

In [ ]:
Phi.display(spher, X4)

In [ ]:
Phi.display()

## Quaternions

In [ ]:
def qprod(p,q):
    if p in R4 and q in R4:
        T1, X1, Y1, Z1 = X4(p)
        T2, X2, Y2, Z2 = X4(q)
        return R4(((T1*T2-X1*X2-Y1*Y2-Z1*Z2).simplify_full(),
                   (T1*X2+X1*T2+Y1*Z2-Z1*Y2).simplify_full(),
                   (T1*Y2-X1*Z2+Y1*T2+Z1*X2).simplify_full(),
                   (T1*Z2+X1*Y2-Y1*X2+Z1*T2).simplify_full()))
    if p in S3 and q in S3:
        a = qprod(Phi(p),Phi(q))
        if X4(a) != (-1,0,0,0):
            return ProjN(R4N(a))
        return ProjS(R4S(a))
    raise ValueError("Cannot evaluate qprod of {} and {}".format(p,q))

In [ ]:
One = S3((0,0,0), chart=stereoN, name='1', latex_name=r'\mathbf{1}')
X4(Phi(One))

In [ ]:
One == S

In [ ]:
minusOne = S3((0,0,0), chart=stereoS, name='-1', latex_name=r'-\mathbf{1}')
X4(Phi(minusOne))

In [ ]:
minusOne == N

In [ ]:
I = S3((1,0,0), chart=stereoN, name='i', latex_name=r'\mathbf{i}')
X4(Phi(I))

In [ ]:
stereoS(I)

In [ ]:
J = S3((0,1,0), chart=stereoN, name='j', latex_name=r'\mathbf{j}')
X4(Phi(J))

In [ ]:
stereoS(J)

In [ ]:
spher(J)

In [ ]:
K = S3((0,0,1), chart=stereoN, name='k', latex_name=r'\mathbf{k}')
X4(Phi(K))

In [ ]:
stereoS(K)

In [ ]:
all([qprod(One,One) == One, qprod(I,I) == minusOne,
     qprod(J,J) == minusOne, qprod(K,K) == minusOne])

In [ ]:
qprod(I, qprod(J,K)) == minusOne

In [ ]:
all([qprod(I,J) == K, qprod(J,K) == I,
     qprod(K,I) == J])

In [ ]:
def qconj(p):
    if p in R4:
        T, X, Y, Z = X4(p)
        return R4((T, -X, -Y, -Z))
    if p in S3:
        a = qconj(Phi(p))
        if X4(a) != (-1,0,0,0):
            return ProjN(a)
        return ProjS(a)
    raise ValueError("Cannot evaluate qconf of {}".format(p)) 

In [ ]:
minusI = qprod(minusOne, I)
Phi(minusI).coord()

In [ ]:
minusJ = qprod(minusOne, J)
Phi(minusJ).coord()

In [ ]:
minusK = qprod(minusOne, K)
Phi(minusK).coord()

In [ ]:
all([qconj(One) == One, 
     qconj(I) == minusI,
     qconj(J) == minusJ, 
     qconj(K) == minusK])

In [ ]:
assume(a != 0)
p = S3((a,b,c), chart=stereoN)
stereoN(qconj(p))

In [ ]:
p = S3((a,b,c), chart=stereoS)
stereoS(qconj(p))

In [ ]:
forget(a!=0)

In [ ]:
def qnorm(p):
    if p in R4:
        T, X, Y, Z = X4(p)
        return (sqrt(T^2 + X^2 + Y^2 + Z^2)).simplify_full()
    if p in S3:
        return 1
    raise ValueError("Cannot evaluate qnorm of {}".format(p)) 

In [ ]:
var('d', domain='real')
p = R4((a,b,c,d))
qnorm(p)

In [ ]:
R4((qnorm(p)^2,0,0,0)) == qprod(qconj(p), p)

In [ ]:
(qnorm(One), qnorm(I), qnorm(J), qnorm(K)) == (1, 1, 1, 1)

In [ ]:
S3.atlas()

In [ ]:
len(S3.atlas())

In [ ]:
S3.top_charts()

## Lie group structure

### Left translations

In [ ]:
p = R4((T,X,Y,Z))
coord_LIp = X4(qprod(Phi(I), p))
coord_LIp

In [ ]:
LI_R4 = R4.diff_map(R4, coord_LIp)
LI_R4.display()

In [ ]:
LI_S3_R4 = LI_R4 * Phi
LI_S3_R4.display()

In [ ]:
UI = U.open_subset('U_I', coord_def={stereoN: (x!=1, y!=0, z!=0)})

In [ ]:
LI_UI = ProjN * LI_S3_R4.restrict(UI, subcodomain=R4N)
LI_UI.display()

In [ ]:
VmI = V.open_subset('V_mI', latex_name=r"V_{-I}", 
                    coord_def={stereoS: (xp!=-1, yp!=0, zp!=0)})
LI_VmI = ProjS * LI_S3_R4.restrict(VmI, subcodomain=R4S)
LI_VmI.display()

In [ ]:
S3.declare_union(UI, VmI)

In [ ]:
LI = S3.diff_map(S3, name='L_I')
LI.add_expression(stereoN.restrict(UI), stereoN, 
                  LI_UI.expr(stereoN.restrict(UI), stereoN))
LI.add_expression(stereoS.restrict(VmI), stereoS, 
                  LI_VmI.expr(stereoS.restrict(VmI), stereoS))

In [ ]:
LI.display(stereoN.restrict(UI), stereoN)

In [ ]:
LI.display(stereoS.restrict(VmI), stereoS)

In [ ]:
all([LI(One)==I, LI(minusOne)==minusI, 
     LI(I)==minusOne, LI(minusI)==One,
     LI(J)==K, LI(minusJ)==minusK,
     LI(K)==minusJ, LI(minusK)==J])

In [ ]:
#def left(p):
#    if p not in S3:
#        raise ValueError("{} is not an element of S^3".format(p))
#    if not hasattr(p, '_left_translation'):
#        print("left trans. of {} created".format(p))
#        p._left_translation = S3.diff_map(S3)
#        p._left_translation._call_ = lambda q : qprod(p, q)
#    return p._left_translation

In [ ]:
LI_R4.display()

In [ ]:
EI = R4.vector_field(name='E_I')
EI[:] = LI_R4.expression()
EI.display()

In [ ]:
eU = stereoN.frame()
eU

In [ ]:
vU = [Phi.restrict(U).pushforward(eU[i]) for i in S3.irange()]
vU

In [ ]:
print(vU[0])

In [ ]:
vU[0].display()

In [ ]:
vU[1].display()

In [ ]:
vU[2].display()

In [ ]:
p = U((x,y,z), chart=stereoN, name='p')
EIp = EI.at(Phi(p))
EIp[:]

In [ ]:
eqs = [(a*vU[0][i] + b*vU[1][i] + c*vU[2][i]).expr() == EIp[i] for i in R4.irange()]
eqs

In [ ]:
sol = solve(eqs, (a,b,c), solution_dict=True)
sol

In [ ]:
e = S3.vector_frame('e')
e

In [ ]:
e[1][stereoN.frame(),:] = (sol[0][a], sol[0][b], sol[0][c])
e[1].display(stereoN.frame())

In [ ]:
test = Phi.restrict(U).pushforward(e[1].restrict(U))
test.display()

In [ ]:
EI.display()